## Load model

In [1]:
import torch
from dataloader import *
from utils import *
import matplotlib.pyplot as plt
from compare_distribution import *
import numpy as np
import os
from models.vgg import VGG11

In [2]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
model = load_model('checkpoints/model.th', device)

In [3]:
model

VGG11(
  (conv1): Sequential(
    (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU(inplace=True)
    (3): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (conv2): Sequential(
    (0): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU(inplace=True)
    (3): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (conv3): Sequential(
    (0): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU(inplace=True)
  )
  (conv4): Sequential(
    (0): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, 

## Load data

In [5]:
dog = torch.load('./data/dog.pt')
dog_data = dog.data
dog_data = np.moveaxis(dog_data, -1, 1)
dog_data = torch.from_numpy(dog_data)
dog_data = dog_data.type(torch.FloatTensor)

In [6]:
cat = torch.load('./data/cat.pt')
cat_data = cat.data
cat_data = np.moveaxis(cat_data, -1, 1)
cat_data = torch.from_numpy(cat_data)
cat_data = cat_data.type(torch.FloatTensor)

In [13]:
cat_adv_fgsm = torch.load('./data/adv_cat_fgsm.pt')

In [14]:
cat_adv_deepfool = torch.load('./data/adv_cat_deepfool.pt')

In [90]:
cat_adv_cw = torch.load('./data/adv_cat_cw.pt')

In [82]:
cat_adv_jsma = torch.load('./data/adv_cat_jsma.pt')

In [91]:
len(cat_adv_cw)

19

## Calculate KL divergence

In [92]:
dog_out = batch_model_execute(model, layer_ct=9, data=dog_data, device=device)
cat_out = batch_model_execute(model, layer_ct=9, data=cat_data, device=device)
adv_cat_out_fgsm = batch_model_execute(model, layer_ct=9, data=cat_adv_fgsm, device=device)
adv_cat_out_df = batch_model_execute(model, layer_ct=9, data=cat_adv_deepfool, device=device)
adv_cat_out_jsma = batch_model_execute(model, layer_ct=9, data=cat_adv_jsma, device=device)
adv_cat_out_cw = batch_model_execute(model, layer_ct=9, data=cat_adv_cw, device=device)

In [24]:
len(adv_cat_out_fgsm)

986

In [25]:
len(adv_cat_out_df)

1114

In [85]:
len(adv_cat_out_jsma)

700

In [16]:
abs_divergence = []
for nr in range(dog_out.shape[1]):
    act_dog = dog_out[:, nr, ...].flatten()
    act_cat = cat_out[:, nr, ...].flatten()
    diverge = kl_divergence(act_dog.numpy(), act_cat.numpy())
    abs_divergence.append(abs(diverge))

In [17]:
np.sort(abs_divergence)[::-1][:10]

array([99.8820624 , 57.11282395,  4.01803332,  1.3264057 ,  1.07477007,
        0.4149808 ,  0.24966941,  0.24875865,  0.24686398,  0.14147899])

In [18]:
abs_divergence = []
for nr in range(cat_out.shape[1]):
    act_cat_adv_fgsm = adv_cat_out_fgsm[:, nr, ...].flatten()
    act_cat = cat_out[:, nr, ...].flatten()
    diverge = kl_divergence(act_cat_adv_fgsm.numpy(), act_cat.numpy())
    abs_divergence.append(abs(diverge))

In [19]:
np.sort(abs_divergence)[::-1][:10]

array([141.43741724, 140.08008976, 135.22436168,  64.19999517,
        61.87704334,  59.96241456,  56.78395501,  54.21964652,
        52.36689503,  51.50964778])

In [20]:
abs_divergence = []
for nr in range(cat_out.shape[1]):
    act_cat_adv_df = adv_cat_out_df[:, nr, ...].flatten()
    act_cat = cat_out[:, nr, ...].flatten()
    diverge = kl_divergence(act_cat_adv_df.numpy(), act_cat.numpy())
    abs_divergence.append(abs(diverge))

In [21]:
np.sort(abs_divergence)[::-1][:10]

array([149.42356987, 134.77685622,  80.7573002 ,  75.39352389,
        69.34133617,  56.12744008,  54.38516406,  51.86348477,
        51.29603692,  44.77712343])

In [86]:
abs_divergence = []
for nr in range(cat_out.shape[1]):
    act_cat_adv_jsma = adv_cat_out_jsma[:, nr, ...].flatten()
    act_cat = cat_out[:, nr, ...].flatten()
    diverge = kl_divergence(act_cat_adv_jsma.numpy(), act_cat.numpy())
    abs_divergence.append(abs(diverge))

In [87]:
np.sort(abs_divergence)[::-1][:10]

array([177.98685808, 146.09583066, 136.31400897, 125.35219135,
       123.83196536, 108.85753191, 108.1113143 ,  75.68121395,
        71.0926223 ,  67.62794263])

In [93]:
abs_divergence = []
for nr in range(cat_out.shape[1]):
    act_cat_adv_cw = adv_cat_out_cw[:, nr, ...].flatten()
    act_cat = cat_out[:, nr, ...].flatten()
    diverge = kl_divergence(act_cat_adv_cw.numpy(), act_cat.numpy())
    abs_divergence.append(abs(diverge))

In [94]:
np.sort(abs_divergence)[::-1][:10]

array([163.42365137, 152.15383536, 144.78956684, 130.02089518,
       125.85270045, 110.27219974, 100.74228241,  96.18598871,
        95.03634067,  93.18001307])

- Divergence between adversarial attacks

In [22]:
abs_divergence = []
for nr in range(cat_out.shape[1]):
    act_cat_adv_df = adv_cat_out_df[:, nr, ...].flatten()
    act_cat_adv_fgsm = adv_cat_out_fgsm[:, nr, ...].flatten()
    diverge = kl_divergence(act_cat_adv_df.numpy(), act_cat_adv_fgsm.numpy())
    abs_divergence.append(abs(diverge))

In [23]:
np.sort(abs_divergence)[::-1][:10]

array([0.86820515, 0.80963881, 0.7563966 , 0.751757  , 0.72145569,
       0.72122479, 0.6936098 , 0.63065694, 0.61427395, 0.61417784])

In [88]:
abs_divergence = []
for nr in range(cat_out.shape[1]):
    act_cat_adv_df = adv_cat_out_df[:, nr, ...].flatten()
    act_cat_adv_jsma = adv_cat_out_jsma[:, nr, ...].flatten()
    diverge = kl_divergence(act_cat_adv_df.numpy(), act_cat_adv_jsma.numpy())
    abs_divergence.append(abs(diverge))

In [89]:
np.sort(abs_divergence)[::-1][:10]

array([1.65400401, 1.36366183, 1.23483498, 1.19153064, 1.11098124,
       1.07574134, 1.03529242, 0.99717403, 0.89416937, 0.88294176])

In [95]:
abs_divergence = []
for nr in range(cat_out.shape[1]):
    act_cat_adv_df = adv_cat_out_df[:, nr, ...].flatten()
    act_cat_adv_cw = adv_cat_out_cw[:, nr, ...].flatten()
    diverge = kl_divergence(act_cat_adv_df.numpy(), act_cat_adv_cw.numpy())
    abs_divergence.append(abs(diverge))

In [96]:
np.sort(abs_divergence)[::-1][:10]

array([5.10961035, 1.06696186, 1.03205453, 0.95606473, 0.84709639,
       0.8466234 , 0.77678929, 0.69604789, 0.69310573, 0.67013513])